In [ ]:
import numpy as np
import pandas as pd
import os
pd.set_option("display.max_rows", None)
import plotly.express as px
#from self_driving_lab_demo.utils.plotting import plot_and_save
import sys
sys.path.append('../utilities')
import simulation_utils as sim
import plotting_utils as plotting
import matplotlib.pyplot as plt
from smt.sampling_methods import LHS
import matplotlib.patches as patches
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from smt.utils.design_space import (
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)
from smt.applications.mixed_integer import MixedIntegerSamplingMethod


In [ ]:
bounds = {"radius": [90, 265], "thickness": [1,20], "npanels": [4,360], "theta": [0,90], "length": [1,150] }
num_samples = len(bounds)*10
num_samples_1 = len(bounds)*1
num_samples_2 = len(bounds)*2
num_digits = 2

In [ ]:
from numpy.random import default_rng

def get_random_samples(bounds, num_samples=9, seed=None):
    rng = default_rng(seed)
    samples = {}
    for parameter, bound in bounds.items():
        samples[parameter] = rng.uniform(bound[0], bound[1], num_samples)
        samples[parameter] = np.round(samples[parameter],num_digits)
    return pd.DataFrame(samples)

def get_random_samples_extended(bounds, num_samples=10, seed=None):
    random_samples_1 = get_random_samples(bounds, num_samples,seed=seed)
    random_samples_2 = get_random_samples(bounds, num_samples*2,seed=seed)
    return pd.concat([random_samples_1,random_samples_2],ignore_index=True,sort=False)

random_samples = get_random_samples_extended(bounds, num_samples,seed=None)
random_samples

In [ ]:
# https://community.plotly.com/t/rotating-3d-plots-with-plotly/34776/2
# https://community.plotly.com/t/how-to-export-animation-and-save-it-in-a-video-format-like-mp4-mpeg-or/64621/2

def draw_samples_distribution_3D(x,y,z):
    fig = go.Figure(go.Scatter3d(x=x, y=y, z=z, mode="markers"))
    x_eye = -1.25
    y_eye = 2
    z_eye = 1.0

    fig.update_layout(
            title="Animation Test",
            width=600,
            height=600,
            scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
            updatemenus=[
                dict(
                    type="buttons",
                showactive=False,
                y=1,
                x=0.8,
                xanchor="left",
                yanchor="bottom",
                pad=dict(t=45, r=10),
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            dict(
                                frame=dict(duration=5, redraw=True),
                                transition=dict(duration=0),
                                fromcurrent=True,
                                mode="immediate",
                            ),
                        ],
                    )
                ],
            )
        ],
    )


    def rotate_z(x, y, z, theta):
        w = x + 1j * y
        return np.real(np.exp(1j * theta) * w), np.imag(np.exp(1j * theta) * w), z

    frames = []
    #pil_frames = []
    for t in np.arange(0, 3.14, 0.025):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
    fig.frames = frames

    fig.show()

In [ ]:
version='v1.5'
if not os.path.exists(f'out/{version}'):
   os.makedirs(f'out/{version}')

height=300
zpos=42
design=4
labels= ["radius", "theta", "npanels", "thickness", "length"]
bounds = {"radius": [95, 255], "thickness": [1,15], "npanels": [4,360], "theta": [0,45], "length": [1,150] }
num_samples = len(bounds)*60
num_samples_to_add = num_samples*2

In [ ]:
def get_latin_hypercube_samples(bounds, num_samples=10):
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    design_space = DesignSpace(
            [
                FloatVariable(l_bounds[0], u_bounds[0]),
                FloatVariable(l_bounds[1],u_bounds[1]),
                IntegerVariable(l_bounds[2],u_bounds[2]),
                FloatVariable(l_bounds[3],u_bounds[3]),
                FloatVariable(l_bounds[4], u_bounds[4]),
            ]
        )
    sampler = MixedIntegerSamplingMethod(LHS, design_space, criterion="ese", random_state=42)
    samples = sampler(num_samples)
    return pd.DataFrame(samples, columns=list(bounds.keys()))

def get_latin_hypercube_samples_extended(samples, bounds, num_samples_to_add=10):
    l_bounds = [bound[0] for bound in bounds.values()]
    u_bounds = [bound[1] for bound in bounds.values()]
    design_space = DesignSpace(
            [
                FloatVariable(l_bounds[0], u_bounds[0]),
                FloatVariable(l_bounds[1],u_bounds[1]),
                IntegerVariable(l_bounds[2],u_bounds[2]),
                FloatVariable(l_bounds[3],u_bounds[3]),
                FloatVariable(l_bounds[4], u_bounds[4]),
            ]
        )
    sampler = MixedIntegerSamplingMethod(LHS, design_space, criterion="ese", random_state=42)
    samples_continued = sampler.expand_lhs(samples, num_samples_to_add, method="ese")
    return pd.DataFrame(samples_continued, columns=list(bounds.keys()))

latin_hypercube_samples=get_latin_hypercube_samples(bounds,num_samples)
latin_hypercube_samples



In [ ]:
latin_hypercube_fig = px.scatter(
    latin_hypercube_samples, x="radius", y="thickness", width=400, height=400
)
latin_hypercube_fig

In [ ]:
draw_samples_distribution_3D(latin_hypercube_samples["radius"], latin_hypercube_samples["npanels"], latin_hypercube_samples["theta"])

In [ ]:
fig, axs = plt.subplots(len(labels), len(labels),figsize=(9,9))
for i in range(len(labels)):
    for j in range(len(labels)):
        axs[j,i].scatter(latin_hypercube_samples[labels[i]],latin_hypercube_samples[labels[j]])
        if i==0:
            axs[j,i].set_ylabel(labels[j])
        if j==len(labels)-1:
            axs[j,i].set_xlabel(labels[i])


In [ ]:
#latin_hypercube_samples_new=get_latin_hypercube_samples_extended(latin_hypercube_samples.values,bounds,num_samples_to_add)
#latin_hypercube_samples_extended=pd.concat([latin_hypercube_samples,latin_hypercube_samples_new],ignore_index=True,sort=False)
#latin_hypercube_samples_extended
#draw_samples_distribution_3D(latin_hypercube_samples_extended["radius"], latin_hypercube_samples_extended["phi"], latin_hypercube_samples_extended["theta"])

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
lh_array=latin_hypercube_samples.to_numpy()
n_outofrange=0
n_crossings=0
list_crossings=[]
list_outofrange=[]

f = open(f"out/{version}/invalid_samples.txt", "w")#
pdf = PdfPages(f'out/{version}/invalid_samples.pdf')
for idx,v in enumerate(lh_array):
    print(v)
    plotting.draw_moderator_configuration(v)
    points=plotting.get_points(v)
    crossed=plotting.is_crossed(points)

    inner_radius=np.sqrt(points[1][0]**2+points[1][1]**2)
    outer_radius=np.sqrt(points[0][0]**2+points[0][1]**2)


    if outer_radius>265:
        dr=outer_radius-265
        [fig,ax]=plotting.draw_panel_border(v,v[0])
        ax.text(-280, -280, f"{idx}: {np.round(v,2)}; corr r={np.round(v[0]-dr,1)}", fontsize=9)
        fig.show()
        pdf.savefig(fig)
       
        list_outofrange.append(idx)
        n_outofrange+=1
        f.write(f"out of range (outer): {idx} {v} (correction: r={np.round(v[0]-dr,1)})"+ "\n")

    if inner_radius < 90:
        dr=90-inner_radius
        [fig,ax]=plotting.draw_panel_border(v,v[0])
        ax.text(-280, -280, f"{idx}: {np.round(v,2)}; corr r={np.round(v[0]+dr,1)}", fontsize=9)
        fig.show()
        pdf.savefig(fig)


        list_outofrange.append(idx)
        n_outofrange+=1
        f.write(f"out of range (inner): {idx} {v} (correction: r={np.round(v[0]+dr,1)})"+ "\n")
        
    if (crossed==True):
        [fig,ax]=plotting.draw_panel_border(v,v[0])
        ax.text(-280, -280, f"{idx}: {np.round(v,2)}", fontsize=9)
        fig.show()
        pdf.savefig()
        n_crossings+=1
        list_crossings.append(idx)
        f.write(f"crossing: {idx} {v}"+ "\n")
pdf.close()

print(f"crossings: {n_crossings}/{num_samples}",f"{(n_crossings)/num_samples*100}%")
print(f"out of range: {n_outofrange}/{num_samples}",f"{(n_outofrange)/num_samples*100}%")
f.write(f"crossings: {n_crossings}/{num_samples}; {(n_crossings)/num_samples*100}%"+ "\n")
f.write(f"out of range: {n_outofrange}/{num_samples}; {(n_outofrange)/num_samples*100}%" + "\n")
f.write(f"crossing: {list_crossings}"+ "\n")
f.write(f"out of range: {list_outofrange}"+ "\n")
f.close()

In [ ]:
lh_array=latin_hypercube_samples.to_numpy()

for idx,v in enumerate(lh_array):
          sim.print_geant4_macro_new_sample(v, idx, mode='LF')

latin_hypercube_samples.to_csv(f'out/{version}/LF-sim-samples_{version}.csv')
latin_hypercube_samples